In [11]:
import pickle
import re
import string
import os
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import PyPDF2
import numpy as np

In [10]:
ps=PorterStemmer()
with open('tfidf_vectorizer.pkl', 'rb') as file:
    tfidf = pickle.load(file)

In [15]:
with open('tfidf_vectorizer.pkl', 'rb') as file:
    tfidf = pickle.load(file)
    
with open('resume.pkl', 'rb') as file:
    rf = pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    labelencoder = pickle.load(file) 

In [28]:
current_classes = labelencoder.classes_

index = np.where(current_classes == "BUSINESS-DEVELOPMENT")[0][0]

current_classes[index] = "DATA-SCIENTIST/ANALYST"

labelencoder.classes_ = current_classes


In [31]:
current_classes = labelencoder.classes_

index = np.where(current_classes == "INFORMATION-TECHNOLOGY")[0][0]

current_classes[index] = "WEB-DEVELOPMENT"

labelencoder.classes_ = current_classes


In [35]:
current_classes = labelencoder.classes_

index = np.where(current_classes == "CORE-ENGINEERING")[0][0]

current_classes[index] = "MECHANICAL/ELECTRICAL ENGINEERING"

labelencoder.classes_ = current_classes


In [36]:
current_classes = labelencoder.classes_

index = np.where(current_classes == "CONSTRUCTION")[0][0]

current_classes[index] = "CIVIL-ENGINEERING"

labelencoder.classes_ = current_classes


In [37]:
labelencoder.classes_

array(['ACCOUNTANT', 'ADVOCATE', 'AGRICULTURE', 'APPAREL', 'ARTS',
       'AUTOMOBILE', 'AVIATION', 'BANKING', 'BPO',
       'DATA-SCIENTIST/ANALYST', 'CHEF', 'CIVIL-ENGINEERING',
       'CONSULTANT', 'DESIGNER', 'DIGITAL-MEDIA',
       'MECHANICAL/ELECTRICAL ENGINEERING', 'FINANCE', 'FITNESS',
       'HEALTHCARE', 'HR', 'WEB-DEVELOPMENT', 'PUBLIC-RELATIONS', 'SALES',
       'TEACHER'], dtype=object)

In [6]:
def preprocess_text(text):
    text=text.lower()
    
    pattern=re.compile('<.*?>')
    text=pattern.sub(r'',text)
    
    pattern=re.compile(r'https?://\S+|www\.\S+')
    text= pattern.sub(r'',text)
    
    text=text.translate(str.maketrans('','',string.punctuation))
    
    new_text=[]
    for words in text.split():
        if words in stopwords.words("english"):
            new_text.append('')
        else:
            new_text.append(words)
    text=" ".join(new_text)
    
    text=" ".join(ps.stem(word)for word in text.split())
    
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [12]:
def pdf_to_text(filepath):
    with open(filepath,"rb") as file:
        reader=PyPDF2.PdfReader(file)
        text=""
        for page in range(len(reader.pages)):
            text+=reader.pages[page].extract_text()
    return text        

In [26]:
def predict_role(filepath):
    text=pdf_to_text(filepath)
    text=[text]
    text=tfidf.transform(text)
    probabilities=rf.predict_proba(text)[0]
    top_1_roles=np.argsort(probabilities)
    top_1_roles=top_1_roles[-1:]
    top_1_roles=np.reshape(top_1_roles,(1,1))
    roles=[]
    for i in top_1_roles:
        roles.append(labelencoder.inverse_transform(i))
    roles=np.flip(roles)
            
    return roles[0][0]

In [38]:
def folder_to_file(folder_path):
    file_paths=[]
    for file_path in os.listdir(folder_path):
        file_paths.append(os.path.join(folder_path,file_path))
    return file_paths  

In [58]:
def screen_resume(folder_path,role,top_n_candidates=5):
    role=labelencoder.transform([role])
    probabilities_map={}
    file_paths=folder_to_file(folder_path)
    for file_path in file_paths:
      text=pdf_to_text(file_path)
      text=preprocess_text(text)
      text=[text]
      text=tfidf.transform(text)
      probabilities=rf.predict_proba(text)[0]
      role_probability=probabilities[role]
      probabilities_map[file_path]=role_probability
    sorted_resumes=sorted(probabilities_map.items(),key=lambda item:item[1],reverse=True)
    resume_links = [item[0] for item in sorted_resumes]
    return resume_links[:top_n_candidates]

In [60]:
screen_resume(r"c:\Users\hp\Desktop\pdf_paths","DATA-SCIENTIST/ANALYST",3)

['c:\\Users\\hp\\Desktop\\pdf_paths\\My_Resume.pdf',
 'c:\\Users\\hp\\Desktop\\pdf_paths\\9ffde1a4-8d10-44ca-af8d-7dd38fd0c50d (1).pdf',
 'c:\\Users\\hp\\Desktop\\pdf_paths\\Resume 2.pdf']

In [61]:
predict_role(r"C:\Users\hp\Desktop\pdf_paths\Resume 2.pdf")

'DATA-SCIENTIST/ANALYST'